# Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Debemos averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

En el informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

### Propósito del proyecto:
**Prueba de hipótesis:**
Averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo.

## Abre el archivo de datos y miro la información general.


In [ ]:
# Cargar todas las librerías
import pandas as pd

# Carga los datos
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [ ]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
# vamos a mostrar las primeras filas N
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


**Observaciones:**
* Columnas en negativo (**days_employed**),
* Datos faltantes en las columnas **days_employed y total_income**,
* Datos con contenido en mayúsculas (**education**).

Se puede observar que los datos de la columna **days_employed** tienen datos negativos y creo que eso traerá problemas a la hora de calcular los datos.

In [ ]:
# Obtener información sobre los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Valores ausentes encontrados:**
- La cantidad total de filas es de 21525
- Hay valores ausentes en 2 columnas (**days_employed** y **total_income**), ambas columnas solo tienen 19351 filas

In [ ]:
#Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
valores_ausentes = df[df['days_employed'].isnull()]
valores_ausentes.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing



Aparentemente los valores parecen simétricos, coinciden las mismas columnas con valores ausentes, pero no podemos estar seguros de está suposición, deberemos realizar investigacioines adicionales y contar los valores ausentes en todas las filas con valores ausentes, para confirmar que las muestras que faltan son del mismo tamaño.
Probemos con otro método.

In [ ]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
print("El número de valores ausentes de las columnas '(Filas, columnas)':")
valores_ausentes = df[df['days_employed'].isnull()]
print("- Experiencia laboral en días:", valores_ausentes.shape)
valores_ausentes = df[df['total_income'].isnull()]
print("- Ingreso mensual: ", valores_ausentes.shape)

El número de valores ausentes de las columnas '(Filas, columnas)':
- Experiencia laboral en días: (2174, 12)
- Ingreso mensual:  (2174, 12)


**Conclusión intermedia**

[¿El número de filas en la tabla filtrada coincide con el número de valores ausentes? ¿Qué conclusión podemos sacar de esto?]

[Calcula el porcentaje de los valores ausentes en comparación con el conjunto de datos completo. ¿Se trata de una porción de datos considerablemente grande? Si es así, es posible que quieras completar los valores ausentes. Para hacer eso, primero debemos definir si los datos ausentes podrían deberse a la característica específica del cliente, como el tipo de empleo u otra cosa. Tendrás que decidir qué característica, según *tú*, podría ser la razón. En segundo lugar, debemos verificar si los valores ausentes dependen de alguna manera del valor de otros indicadores con las columnas con características de clientes, específicas e identificadas.]

[Explica tus próximos pasos y cómo se correlacionan con las conclusiones que has hecho hasta ahora.]

In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
datos_totales = 21525 # Número de datos totales
datos_ausentes_dias_trabajados = 2174 # Número de datos ausentes
datos_conversion = datos_ausentes_dias_trabajados / datos_totales

print(f'Tasa de conversión de datos ausentes es: {datos_conversion:.0%}')


Tasa de conversión de datos ausentes es: 10%


In [ ]:
# Comprobación de la distribución

# unique_income_type = df['income_type'].unique()
# print(unique_income_type)

print(df.isna().sum())
trabajos = df.groupby('income_type')['education_id'].count()
print(trabajos)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
income_type
business                        5085
civil servant                   1459
employee                       11119
entrepreneur                       2
paternity / maternity leave        1
retiree                         3856
student                            1
unemployed                         2
Name: education_id, dtype: int64


Lo que puedo encontrar es la corroboracion que la cantidad de datos ausentes son las mismas en los campos de **days_employed** y también en **total_income**

Por lo que considero que están asociados en ambos casos, es decir:
- Que ambos campos están ligados, si no existe días trabajados tampoco hay monto registrado del ingreso total.
- Por lo que esos datos pueden ser reemplazados en base a un promedio de acuerdo a la totalidad de los datos.

[Empecemos a comprobar si los valores ausentes son aleatorios.]

In [ ]:
# Comprobando la distribución en el conjunto de datos entero
df['income_type'].value_counts(normalize=True)


employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

**Conclusión intermedia**

[¿Es similar la distribución en el conjunto de datos original a la distribución de la tabla filtrada? ¿Qué significa eso para nosotros?]

[Si crees que aún no podemos llegar a ninguna conclusión, investiguemos más a fondo nuestro conjunto de datos. Pensemos en otras razones que podrían llevar a la ausencia de datos y verifiquemos si podemos encontrar algún patrón que nos haga pensar que los valores ausentes no son aleatorios. Ya que es tu trabajo, esta sección es opcional.]

In [ ]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
valores_ausentes['income_type'].value_counts(normalize=True)


employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

**Conclusión intermedia**

[¿Podemos finalmente confirmar que los valores ausentes son accidentales? Verifica cualquier otra cosa que creas que podría ser importante aquí.]

In [ ]:
# Comprobación de otros patrones: explica cuáles
print(df.describe())
valores_ausentes.describe()

           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21525.000000  21525.000000   19351.000000  
mean           0.972544      0.080883   26787.568355  
std            1.420324      0.272661   16475.450632  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   16488.504500  
50%            0.000000      0.000000   23202.870000  
75%            1.000

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


**Conclusiones**

[¿Encontraste algunos patrones? ¿Cómo llegaste a esta conclusión?]

[Explica cómo abordarás los valores ausentes. Ten en cuenta las categorías en las que faltan valores.]

[Planifica brevemente tus próximos pasos en la transformación de datos. Probablemente tendrás que abordar diferentes tipos de problemas: duplicados, diferentes registros, artefactos incorrectos y valores ausentes.]

## Transformación de datos

[Repasemos cada columna para ver qué problemas podemos tener en ellas.]

[Comienza con la eliminación de duplicados y la corrección de la información educativa si es necesario.]

In [ ]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
#dropDuplicate

In [ ]:
# Arregla los registros si es necesario


In [ ]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido



[Comprueba los datos de la columna `children`]

In [ ]:
# Veamos la distribución de los valores en la columna `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

[¿Hay cosas extrañas en esta columna? Si es así, ¿cuál es el porcentaje de datos problemáticos? ¿Cómo es posible que hayan ocurrido? Toma una decisión sobre lo que harás con estos datos y explica tu razonamiento.]

In [ ]:
# [arregla los datos según tu decisión]
typeOf

In [ ]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

#df['children'].value_counts() despues de aplicar los cambios

[Comprueba los datos en la columna `days_employed`. En primer lugar, piensa qué tipo de problemas podría haber, qué posiblemente desees comprobar y cómo lo harás.]

In [ ]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
df['days_employed'].describe()
# analizar los tipos de datos, porque los datos son muy grandes o negativos

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

[Si la cantidad de datos problemáticos es alta, podría deberse a problemas técnicos. Puede que queramos proponer la razón más obvia por la que podría haber sucedido y cuáles podrían haber sido los datos correctos, ya que no podemos eliminar estas filas problemáticas.]

In [ ]:
# Aborda los valores problemáticos, si existen.



In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado


Ahora echemos un vistazo a la edad de clientes para ver si hay algún problema allí. Una vez más, piensa qué datos pueden ser extraños en esta columna, es decir, qué dato no puede ser la edad de alguien.]

In [ ]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

[Decide qué harás con los valores problemáticos y explica por qué.]

In [ ]:
# Resuelve los problemas en la columna `dob_years`, si existen


In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado


[Ahora revisemos la columna `family_status`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar.]

In [ ]:
# Veamos los valores de la columna



In [ ]:
# Aborda los valores problemáticos en `family_status`, si existen



In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado


[Ahora revisemos la columna `gender`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar]

In [ ]:
# Veamos los valores en la columna

In [ ]:
# Aborda los valores problemáticos, si existen

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado



[Ahora vamos a revisar la columna `income_type`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar]

In [ ]:
# Veamos los valores en la columna

In [ ]:
# Aborda los valores problemáticos, si existen

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado



[Ahora veamos si hay duplicados en nuestros datos. Si los hay, tendrás que decidir qué harás con ellos y explicar por qué.]

In [ ]:
# Comprobar los duplicados



In [ ]:
# Aborda los duplicados, si existen

In [ ]:
# Última comprobación para ver si tenemos duplicados


In [ ]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones

[Describe tu nuevo conjunto de datos: di brevemente qué has cambiado y cuál es el porcentaje de cambios, si hubo alguno.]


# Trabajar con valores ausentes

[Para acelerar el trabajo con algunos datos, puede que necesites trabajar con diccionarios para algunos valores, en los que se proporcionan IDs. Explica por qué y con qué diccionarios vas a trabajar.]

In [ ]:
# Encuentra los diccionarios
# education_id / education 1 a titulo., 2 a bachillerato,.../ en todos los que tenga _id

### Restaurar valores ausentes en `total_income`

[Indica brevemente qué columnas tienen valores ausentes que debes abordar. Explica cómo las arreglarás.]


[Empieza por abordar los valores ausentes del ingreso total. Crea una categoría de edad para los clientes. Crea una nueva columna con la categoría de edad. Esta estrategia puede ayudar a calcular valores para el ingreso total.]


In [ ]:
# Vamos a escribir una función que calcule la categoría de edad
# crear funcion y categorizar primero


In [ ]:
# Prueba si la función funciona bien


In [ ]:
# Crear una nueva columna basada en la función



In [ ]:
# Comprobar cómo los valores en la nueva columna



[Piensa en los factores de los que suelen depender los ingresos. Eventualmente, tendrás que averiguar si debes usar valores medios o medianos para reemplazar los valores ausentes. Para tomar esta decisión, probablemente querrás ver la distribución de los factores que, según tu análisis, pueden tener un impacto en los ingresos de uno.]

[Crea una tabla que solo tenga datos sin valores ausentes. Estos datos se utilizarán para restaurar los valores ausentes.]

In [ ]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
# filtro / utilizar agrupamimento, por ejm por edad y ver su salario promedio

In [ ]:
# Examina los valores medios de los ingresos en función de los factores que identificaste

In [ ]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste


[Repite tales comparaciones para múltiples factores. Asegúrate de considerar diferentes aspectos y explica tu razonamiento.]



[Determina qué características definen mejor los ingresos y decide si utilizarás una mediana o una media. Explica por qué tomaste esta decisión.]


In [ ]:
#  Escribe una función que usaremos para completar los valores ausentes
# reemplazar en funcion a otras categoris (18-30) mas o menos no solo de la edad


In [ ]:
# Comprueba si funciona


In [ ]:
# Aplícalo a cada fila
#funcion aplicando .update()

In [ ]:
# Comprueba si tenemos algún error


[Si has encontrado errores al preparar los valores para los datos ausentes, probablemente signifique que hay algo especial en los datos de la categoría. Piénsalo un poco: tal vez hará falta arreglar algunas cosas manualmente, si hay suficientes datos para encontrar medianas/medias.]


In [ ]:
# Reemplazar los valores ausentes si hay algún error


[Cuando creas que has terminado con `total_income`, comprueba que el número total de valores en esta columna coincida con el número de valores en otras columnas.]

In [ ]:
# Comprobar el número de entradas en las columnas



###  Restaurar valores en `days_employed`

[Piensa en los parámetros que pueden ayudarte a restaurar los valores ausentes en esta columna. Eventualmente, tendrás que averiguar si debes usar valores medios o medianos para reemplazar los valores ausentes. Probablemente llevarás a cabo una investigación similar a la que realizaste cuando restauraste los datos en la columna anterior.]

In [ ]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados




In [ ]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

[Decide qué vas a utilizar: medias o medianas. Explica por qué.]

In [ ]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado


In [ ]:
# Comprueba que la función funciona



In [ ]:
# Aplicar la función al income_type



In [ ]:
# Comprueba si la función funcionó



In [ ]:
# Reemplazar valores ausentes



[Cuando creas que has terminado con `total_income`, comprueba que el número total de valores en esta columna coincida con el número de valores en otras columnas.]

In [ ]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes

## Clasificación de datos

[Para poder responder a las preguntas y probar las diferentes hipótesis, querrás trabajar con datos clasificados. Mira las preguntas formuladas que debes responder. Piensa qué parte de los datos tiene que ser clasificada para responder a estas preguntas. A continuación, encontrarás una plantilla a través de la cual puedes trabajar para clasificar los datos. El primer procesamiento paso a paso cubre los datos de texto; el segundo aborda los datos numéricos que necesitan ser clasificados. Puedes usar ambas o ninguna de las instrucciones sugeridas, eso solo depende de ti.]

[Independientemente de cómo decidas abordar la clasificación, asegúrate de proporcionar una explicación clara de la razón por la que tomaste tu decisión. Recuerda: este es tu trabajo y aquí tú tomas todas las decisiones.]


In [ ]:
# Muestra los valores de los datos seleccionados para la clasificación



[Vamos a comprobar los valores únicos]

In [ ]:
# Comprobar los valores únicos

[¿Qué grupos principales puedes identificar en función de los valores únicos?]

[Según estos temas, probablemente querremos clasificar nuestros datos.]


In [ ]:
# Escribamos una función para clasificar los datos en función de temas comunes


In [ ]:
# Crea una columna con las categorías y cuenta los valores en ellas



[Si decides clasificar los datos numéricos, también tendrás que crear las categorías para ello.]

In [ ]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación


In [ ]:
# Obtener estadísticas resumidas para la columna



[Decide qué rangos utilizarás para agrupar y explica por qué.]

In [ ]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos



In [ ]:
# Crear una columna con categorías


In [ ]:
# Contar los valores de cada categoría para ver la distribución


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
# Comprueba los datos sobre los hijos y los pagos puntuales


# Calcular la tasa de incumplimiento en función del número de hijos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
# Comprueba los datos del estado familiar y los pagos a tiempo



# Calcular la tasa de incumplimiento basada en el estado familiar



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo



# Calcular la tasa de incumplimiento basada en el nivel de ingresos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]


# Conclusión general

[Enumera tus conclusiones en esta última sección. Asegúrate de incluir todas las conclusiones importantes que hiciste y que te llevaron a la forma en que procesaste y analizaste los datos. Habla de los valores ausentes, los duplicados y las posibles razones y soluciones para los artefactos problemáticos que tuviste que abordar.]

[Enumera tus conclusiones con respecto a las preguntas planteadas aquí también.]
